# Análise de Performance e Qualidade dos Dados SINAN

Este notebook analisa:

- Tempo de carregamento dos dados
- Uso de memória
- Colunas disponíveis e suas características
- Gargalos de performance
- Qualidade dos dados (valores nulos, tipos, etc.)


In [ ]:
import pandas as pd
import numpy as np
import time
import psutil
import os
from pathlib import Path
import sys

# Adicionar o diretório raiz ao path para importar módulos
sys.path.append('.')

print("Bibliotecas importadas com sucesso!")


## 1. Análise de Memória e Performance Inicial


In [ ]:
def get_memory_usage():
    """Retorna o uso de memória atual em MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def format_bytes(bytes_size):
    """Formata bytes para formato legível"""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024.0:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024.0
    return f"{bytes_size:.2f} PB"

memoria_inicial = get_memory_usage()
print(f"Memória inicial: {memoria_inicial:.2f} MB")


## 2. Carregar Dados e Medir Performance


In [ ]:
# Encontrar arquivos Parquet
parquet_dir = Path('data/raw/VIOLBR-PARQUET')
arquivos_parquet = list(parquet_dir.glob('*.parquet'))

print(f"Arquivos Parquet encontrados: {len(arquivos_parquet)}")
for arquivo in arquivos_parquet:
    tamanho = arquivo.stat().st_size
    print(f"  - {arquivo.name}: {format_bytes(tamanho)}")


In [ ]:
# Carregar primeiro arquivo e medir tempo
if arquivos_parquet:
    arquivo_teste = arquivos_parquet[0]
    
    print(f"\nCarregando: {arquivo_teste.name}")
    
    inicio = time.time()
    mem_antes = get_memory_usage()
    
    df = pd.read_parquet(arquivo_teste)
    
    fim = time.time()
    mem_depois = get_memory_usage()
    
    tempo_carregamento = fim - inicio
    memoria_usada = mem_depois - mem_antes
    
    print(f"\n=== RESULTADOS DO CARREGAMENTO ===")
    print(f"Tempo de carregamento: {tempo_carregamento:.2f} segundos")
    print(f"Memória usada: {memoria_usada:.2f} MB")
    print(f"Memória total após carregamento: {mem_depois:.2f} MB")
    print(f"\nShape do DataFrame: {df.shape}")
    print(f"Total de registros: {len(df):,}")
    print(f"Total de colunas: {len(df.columns)}")


## 3. Análise Detalhada das Colunas


In [ ]:
if 'df' in locals():
    print("=== INFORMAÇÕES GERAIS DO DATAFRAME ===")
    print(f"\nColunas ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:3d}. {col}")
    
    print(f"\n\nTipos de dados:")
    tipos = df.dtypes.value_counts()
    for tipo, count in tipos.items():
        print(f"  {tipo}: {count} colunas")


In [ ]:
# Análise detalhada de cada coluna
if 'df' in locals():
    print("\n=== ANÁLISE DETALHADA POR COLUNA ===\n")
    
    analise_colunas = []
    
    for col in df.columns:
        inicio_col = time.time()
        
        # Informações básicas
        tipo = str(df[col].dtype)
        total = len(df)
        nulos = df[col].isna().sum()
        percentual_nulos = (nulos / total * 100) if total > 0 else 0
        
        # Memória usada pela coluna
        memoria_col = df[col].memory_usage(deep=True) / 1024 / 1024  # MB
        
        # Valores únicos (apenas para colunas com poucos valores únicos)
        valores_unicos = df[col].nunique()
        
        tempo_analise = time.time() - inicio_col
        
        analise_colunas.append({
            'Coluna': col,
            'Tipo': tipo,
            'Total': total,
            'Nulos': nulos,
            '% Nulos': f"{percentual_nulos:.2f}%",
            'Valores Únicos': valores_unicos,
            'Memória (MB)': f"{memoria_col:.2f}",
            'Tempo Análise (s)': f"{tempo_analise:.4f}"
        })
    
    df_analise = pd.DataFrame(analise_colunas)
    
    # Ordenar por memória (maiores primeiro)
    df_analise['Memoria_Num'] = df_analise['Memória (MB)'].astype(float)
    df_analise = df_analise.sort_values('Memoria_Num', ascending=False)
    
    print("\nTop 20 colunas que mais consomem memória:")
    display(df_analise[['Coluna', 'Tipo', 'Memória (MB)', '% Nulos', 'Valores Únicos']].head(20))
    
    print("\n\nColunas com mais valores nulos:")
    df_analise['Nulos_Num'] = df_analise['Nulos'].astype(int)
    display(df_analise[['Coluna', 'Nulos', '% Nulos']].sort_values('Nulos_Num', ascending=False).head(20))


## 4. Identificar Gargalos de Performance


In [ ]:
if 'df' in locals():
    print("=== TESTES DE PERFORMANCE ===\n")
    
    # Teste 1: Filtro simples
    print("1. Testando filtro simples (SEXO == 'F'):")
    inicio = time.time()
    if 'SEXO' in df.columns:
        resultado = df[df['SEXO'] == 'F']
        tempo = time.time() - inicio
        print(f"   Tempo: {tempo:.4f}s | Registros encontrados: {len(resultado):,}")
    else:
        print("   Coluna SEXO não encontrada")
    
    # Teste 2: Agrupamento
    print("\n2. Testando agrupamento (por UF):")
    inicio = time.time()
    if 'UF_NOTIFIC' in df.columns:
        resultado = df.groupby('UF_NOTIFIC').size()
        tempo = time.time() - inicio
        print(f"   Tempo: {tempo:.4f}s | Grupos: {len(resultado)}")
    else:
        print("   Coluna UF_NOTIFIC não encontrada")
    
    # Teste 3: Aplicar função
    print("\n3. Testando apply (criar faixa etária):")
    inicio = time.time()
    if 'NU_IDADE_N' in df.columns:
        def get_age_group(idade):
            if pd.isna(idade):
                return 'Não informado'
            idade_int = int(idade) if isinstance(idade, (int, float)) else 0
            if idade_int < 2:
                return '0-1 anos'
            elif idade_int < 6:
                return '2-5 anos'
            elif idade_int < 10:
                return '6-9 anos'
            elif idade_int < 14:
                return '10-13 anos'
            elif idade_int < 18:
                return '14-17 anos'
            else:
                return '18+ anos'
        
        resultado = df['NU_IDADE_N'].apply(get_age_group)
        tempo = time.time() - inicio
        print(f"   Tempo: {tempo:.4f}s | Valores únicos: {resultado.nunique()}")
    else:
        print("   Coluna NU_IDADE_N não encontrada")
    
    # Teste 4: Conversão de data
    print("\n4. Testando conversão de data (DT_NOTIFIC):")
    inicio = time.time()
    if 'DT_NOTIFIC' in df.columns:
        try:
            resultado = pd.to_datetime(df['DT_NOTIFIC'], format='%Y%m%d', errors='coerce')
            tempo = time.time() - inicio
            print(f"   Tempo: {tempo:.4f}s | Datas válidas: {resultado.notna().sum():,}")
        except Exception as e:
            print(f"   Erro: {e}")
    else:
        print("   Coluna DT_NOTIFIC não encontrada")
    
    # Teste 5: Múltiplos filtros
    print("\n5. Testando múltiplos filtros (UF + SEXO + Idade):")
    inicio = time.time()
    filtros = []
    if 'UF_NOTIFIC' in df.columns:
        filtros.append(df['UF_NOTIFIC'] == df['UF_NOTIFIC'].iloc[0] if len(df) > 0 else False)
    if 'SEXO' in df.columns:
        filtros.append(df['SEXO'] == 'F')
    if 'NU_IDADE_N' in df.columns:
        filtros.append(df['NU_IDADE_N'] < 18)
    
    if filtros:
        mask = pd.concat(filtros, axis=1).all(axis=1) if len(filtros) > 1 else filtros[0]
        resultado = df[mask]
        tempo = time.time() - inicio
        print(f"   Tempo: {tempo:.4f}s | Registros encontrados: {len(resultado):,}")
    else:
        print("   Colunas necessárias não encontradas")


In [ ]:
if 'df' in locals():
    print("=== ANÁLISE DE MEMÓRIA ===\n")
    
    memoria_atual = get_memory_usage()
    memoria_df = df.memory_usage(deep=True).sum() / 1024 / 1024  # MB
    
    print(f"Memória total do processo: {memoria_atual:.2f} MB")
    print(f"Memória usada pelo DataFrame: {memoria_df:.2f} MB")
    print(f"Memória adicional (overhead): {memoria_atual - memoria_df:.2f} MB")
    
    # Análise por tipo de dado
    print("\nMemória por tipo de dado:")
    memoria_por_tipo = df.memory_usage(deep=True).groupby(df.dtypes).sum() / 1024 / 1024
    for tipo, memoria in memoria_por_tipo.items():
        print(f"  {tipo}: {memoria:.2f} MB")


## 6. Recomendações de Otimização


In [ ]:
if 'df' in locals() and 'df_analise' in locals():
    print("=== RECOMENDAÇÕES DE OTIMIZAÇÃO ===\n")
    
    recomendacoes = []
    
    # Verificar colunas com muitos nulos
    colunas_muitos_nulos = df_analise[df_analise['Nulos_Num'] > len(df) * 0.9]
    if len(colunas_muitos_nulos) > 0:
        recomendacoes.append({
            'Tipo': 'Qualidade de Dados',
            'Problema': f'{len(colunas_muitos_nulos)} colunas com >90% de valores nulos',
            'Recomendação': 'Considerar remover essas colunas ou usar sparse arrays',
            'Colunas': ', '.join(colunas_muitos_nulos['Coluna'].head(5).tolist())
        })
    
    # Verificar colunas object que poderiam ser category
    colunas_object = df.select_dtypes(include=['object']).columns
    colunas_candidatas_category = []
    for col in colunas_object:
        if df[col].nunique() < len(df) * 0.1:  # Menos de 10% de valores únicos
            colunas_candidatas_category.append(col)
    
    if colunas_candidatas_category:
        recomendacoes.append({
            'Tipo': 'Otimização de Memória',
            'Problema': f'{len(colunas_candidatas_category)} colunas object com poucos valores únicos',
            'Recomendação': 'Converter para category para economizar memória',
            'Colunas': ', '.join(colunas_candidatas_category[:5])
        })
    
    # Verificar colunas numéricas que poderiam ser tipos menores
    colunas_int64 = df.select_dtypes(include=['int64']).columns
    colunas_candidatas_int32 = []
    for col in colunas_int64:
        if df[col].min() >= np.iinfo(np.int32).min and df[col].max() <= np.iinfo(np.int32).max:
            colunas_candidatas_int32.append(col)
    
    if colunas_candidatas_int32:
        recomendacoes.append({
            'Tipo': 'Otimização de Memória',
            'Problema': f'{len(colunas_candidatas_int32)} colunas int64 que cabem em int32',
            'Recomendação': 'Converter para int32 para economizar 50% de memória',
            'Colunas': ', '.join(colunas_candidatas_int32[:5])
        })
    
    # Verificar se DuckDB seria útil
    if len(df) > 1_000_000:
        recomendacoes.append({
            'Tipo': 'Performance',
            'Problema': f'Dataset grande ({len(df):,} registros)',
            'Recomendação': 'Considerar usar DuckDB para queries mais rápidas',
            'Colunas': 'N/A'
        })
    
    if recomendacoes:
        df_recomendacoes = pd.DataFrame(recomendacoes)
        display(df_recomendacoes)
    else:
        print("Nenhuma recomendação específica. Os dados parecem estar bem otimizados!")


## 7. Comparação: Pandas vs DuckDB (se disponível)


In [ ]:
try:
    import duckdb
    
    if 'df' in locals():
        print("=== COMPARAÇÃO PANDAS vs DUCKDB ===\n")
        
        # Teste 1: Filtro simples
        print("1. Filtro simples (SEXO == 'F'):")
        
        # Pandas
        inicio = time.time()
        if 'SEXO' in df.columns:
            resultado_pandas = df[df['SEXO'] == 'F']
            tempo_pandas = time.time() - inicio
            print(f"   Pandas: {tempo_pandas:.4f}s")
        
        # DuckDB
        inicio = time.time()
        if 'SEXO' in df.columns:
            resultado_duckdb = duckdb.sql("SELECT * FROM df WHERE SEXO = 'F'").df()
            tempo_duckdb = time.time() - inicio
            print(f"   DuckDB: {tempo_duckdb:.4f}s")
            print(f"   Ganho: {tempo_pandas/tempo_duckdb:.2f}x mais rápido" if tempo_duckdb > 0 else "   N/A")
        
        # Teste 2: Agrupamento
        print("\n2. Agrupamento (por UF):")
        
        # Pandas
        inicio = time.time()
        if 'UF_NOTIFIC' in df.columns:
            resultado_pandas = df.groupby('UF_NOTIFIC').size()
            tempo_pandas = time.time() - inicio
            print(f"   Pandas: {tempo_pandas:.4f}s")
        
        # DuckDB
        inicio = time.time()
        if 'UF_NOTIFIC' in df.columns:
            resultado_duckdb = duckdb.sql("SELECT UF_NOTIFIC, COUNT(*) as count FROM df GROUP BY UF_NOTIFIC").df()
            tempo_duckdb = time.time() - inicio
            print(f"   DuckDB: {tempo_duckdb:.4f}s")
            print(f"   Ganho: {tempo_pandas/tempo_duckdb:.2f}x mais rápido" if tempo_duckdb > 0 else "   N/A")
        
except ImportError:
    print("DuckDB não está instalado. Para comparar performance, instale com: pip install duckdb")


In [ ]:
if 'df' in locals():
    print("=== RESUMO EXECUTIVO ===\n")
    
    print(f"📊 DADOS:")
    print(f"   - Total de registros: {len(df):,}")
    print(f"   - Total de colunas: {len(df.columns)}")
    print(f"   - Tamanho em memória: {df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
    
    print(f"\n⚡ PERFORMANCE:")
    if 'tempo_carregamento' in locals():
        print(f"   - Tempo de carregamento: {tempo_carregamento:.2f}s")
    if 'memoria_usada' in locals():
        print(f"   - Memória usada: {memoria_usada:.2f} MB")
    
    print(f"\n📈 QUALIDADE:")
    total_nulos = df.isna().sum().sum()
    total_celulas = len(df) * len(df.columns)
    percentual_nulos = (total_nulos / total_celulas * 100) if total_celulas > 0 else 0
    print(f"   - Total de valores nulos: {total_nulos:,} ({percentual_nulos:.2f}%)")
    
    colunas_completas = (df.isna().sum() == 0).sum()
    print(f"   - Colunas sem valores nulos: {colunas_completas}/{len(df.columns)}")
    
    print(f"\n💡 PRINCIPAIS GARGALOS IDENTIFICADOS:")
    if 'df_analise' in locals():
        top_gargalos = df_analise.nlargest(5, 'Memoria_Num')
        for idx, row in top_gargalos.iterrows():
            print(f"   - {row['Coluna']}: {row['Memória (MB)']} MB ({row['% Nulos']} nulos)")
    
    print(f"\n✅ PRÓXIMOS PASSOS:")
    print(f"   1. Revisar recomendações de otimização acima")
    print(f"   2. Considerar usar DuckDB para queries complexas")
    print(f"   3. Avaliar remover colunas com muitos valores nulos")
    print(f"   4. Converter colunas object para category quando apropriado")
